<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Reinforcing_Human_in_the_Loop_with_Active_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np

# Define a simple neural network model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def predict(self, x):
        self.eval()
        with torch.no_grad():
            outputs = self.forward(x)
            probabilities = torch.softmax(outputs, dim=1)
        return probabilities

# Initialize the model
model = SimpleModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Dummy data creation
X = torch.randn(1000, 784)  # Example data (e.g., MNIST flattened)
y = torch.randint(0, 10, (1000,))  # Example labels (e.g., 10 classes)

# Split data into training and unlabeled datasets
X_train, X_unlabeled, y_train, _ = train_test_split(X, y, test_size=0.8, random_state=42)

# DataLoader for training data
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train the model on initial training data
model.train()
for epoch in range(10):  # 10 epochs for initial training
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Assuming a function to get model predictions (probabilities)
def get_predictions(model, data):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        return torch.softmax(outputs, dim=1).numpy()

# Function to calculate uncertainty (entropy)
def get_most_uncertain(predictions, top_n=10):
    entropy = -np.sum(predictions * np.log(predictions + 1e-10), axis=1)
    uncertain_indices = np.argsort(entropy)[-top_n:]
    return uncertain_indices

# Function to simulate querying human labels with predefined labels
def query_human_label(uncertain_samples, predefined_labels):
    human_labels = []
    for i, sample in enumerate(uncertain_samples):
        if i >= len(predefined_labels):
            break
        label = predefined_labels[i]
        print(f"Simulated label for sample {i}: {label}")
        human_labels.append(label)
    return human_labels

# Example predefined labels for simulation
predefined_labels = [1, 0, 2, 1, 0, 1, 2, 1, 0, 1]

# Active learning loop with predefined labels
epochs = 5  # Number of active learning iterations
for epoch in range(epochs):
    # Step 1: Predict on unlabeled data
    predictions = model.predict(X_unlabeled)

    # Step 2: Identify most uncertain samples
    uncertain_indices = get_most_uncertain(predictions.numpy(), top_n=10)
    uncertain_samples = X_unlabeled[uncertain_indices]

    # Step 3: Query human labels for uncertain samples using predefined labels
    human_labels = query_human_label(uncertain_samples, predefined_labels)
    if len(human_labels) < len(uncertain_samples):
        break  # Exit the loop if labeling was stopped by the user

    # Step 4: Train the model on the newly labeled data
    new_labels_tensor = torch.tensor(human_labels)
    new_data_tensor = uncertain_samples[:len(human_labels)]

    # Adding newly labeled data to the training set
    X_train = torch.cat((X_train, new_data_tensor), dim=0)
    y_train = torch.cat((y_train, new_labels_tensor), dim=0)

    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()